# 01 - Interview Question Generation
`Author: Abdlazeez Jimoh`

In [2]:
from openai import OpenAI
import json
import pprint
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
client = OpenAI()

In [4]:
class BaseAgent:
    def __init__(self):
        self.client = OpenAI()
        self.system_prompt = None
        self.user_prompt = None

    def __call__(self, *args, **kwargs):
        raise NotImplementedError


class QuestionGeneratorAgent(BaseAgent):
    def __init__(self):
        super().__init__()
        self.system_prompt = 'You are a non-technical interviewer that generate at least 4 interview questions across the following categories:\n- personal\n- role-specific\n- behavioural\n- situational\n\nMake sure there is a question for each category.\n\nYou answer strictly as single JSON string. Don\'t include any other verbose texts and don\'t include the markdown syntax anywhere.\n\nJSON format:\n[{"question": "<question>", "question_type": "<question_type>"}]'
        self.user_prompt = (
            "Create a list of {n_questions} questions for an interview with {description}."
        )

    def __call__(self, description: str, n_questions: int = 5) -> list[dict[str, str]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 5.

        Returns:
            list: A list of generated interview questions.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def run(self, description: str, n_questions: int = 5) -> list[dict[str, str]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 5.

        Returns:
            list: A list of generated interview questions.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def _generate(self, description: str, n_questions: int) -> list[dict[str, str]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int): The number of questions to generate.

        Returns:
            list: A list of generated interview questions.
        """

        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {
                        "role": "system",
                        "content": self.system_prompt,
                    },
                    {
                        "role": "user",
                        "content": self.user_prompt.format(
                            n_questions=n_questions, description=description
                        ),
                    },
                ],
                temperature=0.5,
                max_tokens=1024,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
            )
            pprint.pprint(json.loads(response.choices[0].message.content or "[]"))

            questions = json.loads(response.choices[0].message.content or "[]")

            return questions
        except Exception as e:
            print(e)
            return []

In [5]:
question_generator = QuestionGeneratorAgent()
questions = question_generator.run(
    "a software engineer at a startup in San Francisco. I have 5 years of experience and I'm looking for a new job."
)

print(questions)

[{'question': 'Tell me about a time when you had to work under a tight '
              'deadline. How did you manage your time and prioritize tasks?',
  'question_type': 'behavioural'},
 {'question': 'Can you describe a challenging technical problem you faced in '
              'your previous role and how you approached solving it?',
  'question_type': 'role-specific'},
 {'question': 'How do you stay updated with the latest technologies and trends '
              'in the software development industry?',
  'question_type': 'personal'},
 {'question': 'Imagine you are working on a project and a team member is '
              'consistently not meeting deadlines. How would you address this '
              'situation?',
  'question_type': 'situational'}]
[{'question': 'Tell me about a time when you had to work under a tight deadline. How did you manage your time and prioritize tasks?', 'question_type': 'behavioural'}, {'question': 'Can you describe a challenging technical problem you faced i